<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Chapter-4---Elements-of-Algebraic-Graph-Theory" data-toc-modified-id="Chapter-4---Elements-of-Algebraic-Graph-Theory-1">Chapter 4 - Elements of Algebraic Graph Theory</a></span></li><li><span><a href="#4.1-The-adjacency-matrix" data-toc-modified-id="4.1-The-adjacency-matrix-2">4.1 The adjacency matrix</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Basic-Graphs,-their-Adjacency-Matrix-and-Spectrum-(Table-4.1)" data-toc-modified-id="Basic-Graphs,-their-Adjacency-Matrix-and-Spectrum-(Table-4.1)-2.0.1">Basic Graphs, their Adjacency Matrix and Spectrum (Table 4.1)</a></span></li></ul></li><li><span><a href="#4.3.2-Connectivity-and-adjacency-powers" data-toc-modified-id="4.3.2-Connectivity-and-adjacency-powers-2.1">4.3.2 Connectivity and adjacency powers</a></span></li><li><span><a href="#4.4.-Graph-theoretical-characterization-of-primitive-matrices" data-toc-modified-id="4.4.-Graph-theoretical-characterization-of-primitive-matrices-2.2">4.4. Graph theoretical characterization of primitive matrices</a></span></li></ul></li></ul></div>

In [1]:
%matplotlib widget

# Import packages
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import sys, os
from matplotlib.colors import ListedColormap
# For interactive graphs
import ipywidgets as widgets

# Import self defined functions
#import ch1_lib  # Chapter 1 specific library
sys.path.insert(1, os.path.join(sys.path[0], '..'))  # Need to call this for importing library from parent folder
import lib  # General library

# Settings
custom_figsize= (6, 4) # Might need to change this value to fit the figures to your screen
custom_figsize_square = (5, 5) 

Chapter 4:
- Figure 4.5: code to generate the adjacency matrix of a grid graph and compute its spectral radius
- simulate/visualize the evolution of the Leslie population model in E4.14

# Chapter 4 - Elements of Algebraic Graph Theory
These Jupyter Notebook scripts contain some examples, visualization and supplements accompanying the book "Lectures on Network Systems" by Francesco Bullo http://motion.me.ucsb.edu/book-lns/. These scripts are published with the MIT license. **Make sure to run the first cell above to import all necessary packages and functions and adapt settings in case.** In this script it is necessary to execute cell by cell chronologically due to reocurring examples (Tip: Use the shortcut Shift+Enter to execute each cell). Most of the functions are kept in separate files to keep this script neat.

# 4.1 The adjacency matrix
First it is shown how to access the weighted adjacency matrix for the example in the script, when the graph is already created in NetworkX:

In [2]:
# Digraph Example from section 3.5 with weights, self_loops etc.
G_di = nx.DiGraph()
edges = [(1,2), (2,1), (2,4), (1,3), (3,5), (5,1), (5,5), (3,4), (5,4)]
weights = [3.7, 8.9, 1.2, 2.6, 2.3, 4.4, 4.4, 1.9, 2.7]
for edge, weight in zip(edges, weights):
    G_di.add_edge(*edge, weight=weight)
pos_di = {1:[0.1,0.2],2:[.4,.5],3:[.5,.2],4:[.8,.5], 5:[.9,.2]}  # Define position of nodes in digraph plot

# Plot first digraph again with weights visualization from section 3.5
fig, ax41 = plt.subplots(figsize=custom_figsize)
nx.draw_networkx(G_di, node_size=100, ax=ax41, pos=pos_di, connectionstyle='arc3, rad = 0.1')
labels = nx.get_edge_attributes(G_di,'weight')
nx.draw_networkx_edge_labels(G_di,pos=pos_di,edge_labels=labels, label_pos=0.2)

# This will always result in a sparse matrix, nodelist argument important to keep order of nodes
A_di = nx.linalg.graphmatrix.adjacency_matrix(G_di, nodelist=range(1, G_di.number_of_nodes()+1)).toarray()
print("Adjacency Matrix determined by Networkx:")
lib.matprint(A_di)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Adjacency Matrix determined by Networkx:
|    0  3.7  2.6    0    0  |
|  8.9    0    0  1.2    0  |
|    0    0    0  1.9  2.3  |
|    0    0    0    0    0  |
|  4.4    0    0  2.7  4.4  |


### Basic Graphs, their Adjacency Matrix and Spectrum (Table 4.1)

Below are further basic graphs and their adjacency matrix and their positive matrix entries in a binary representation similar to Figure 4.2. Further on, the spectrum is plotted an can be compared with the given formula in Table 4.1.

In [3]:
n = 6
# Path Graph
G_path = nx.path_graph(n)
# Cycle Graph
G_cycle = nx.cycle_graph(n)
# Star Graph
G_star = nx.star_graph(n-1)
# Complete Graph
G_complete = nx.complete_graph(n)
# Complete bipartite Graph
G_bipartite = nx.complete_bipartite_graph(n//2, n//2)

all_basic_graphs = {
    "Path Graph": G_path,
    "Cycle Graph": G_cycle,
    "Star Graph":G_star,
    "Complete Graph": G_complete,
    "Complete bipartite Graph": G_bipartite,
    }
spectrums = ["$\{2 cos(\pi i/(n + 1)) | i ∈ \{1, . . . , n\}\}$",
             "$\{2 cos(2\pi i/n)) | i ∈ \{1, . . . , n\}\}$",
             "$\{ \sqrt{n − 1}, 0, . . . , 0, − \sqrt{n − 1}\}$",
             "$\{(n − 1), −1, . . . , −1\}$",
             "$\{\sqrt{nm}, 0, . . . , 0, − \sqrt{nm}\}$"]

def plot_matrix_binary(M, ax, name=''):
    blue_map = ListedColormap(["blue", "white"])
    zeros =  M == 0
    im = ax.imshow(zeros, cmap=blue_map)
    ax.set_xticks([])
    ax.set_yticks([])

In [4]:
# Plotting the graph itself, the binary adjacency matrix visualization with the actual values written inside
fig, axs412 = plt.subplots(len(all_basic_graphs), 3, figsize=(custom_figsize[0]*1.2, custom_figsize[1]*3))
for count, (key, graph) in enumerate(all_basic_graphs.items()):
    if key == "Complete bipartite Graph":
        nx.draw_networkx(graph, node_size=100, ax=axs412[count, 0], pos=nx.drawing.layout.bipartite_layout(graph, list(range(0, n//2))))
    else:
        nx.draw_networkx(graph, node_size=100, ax=axs412[count, 0], connectionstyle='arc3, rad = 0.1')
    A = nx.linalg.graphmatrix.adjacency_matrix(graph, nodelist=range(0, graph.number_of_nodes())).toarray()
    plot_matrix_binary(A, axs412[count, 1])
    axs412[count, 0].set_xlabel(key)
    lib.plot_spectrum(A, axs412[count, 2]);
    axs412[count, 2].set(title=spectrums[count])
fig.subplots_adjust(hspace=0.5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 4.3.2 Connectivity and adjacency powers
In this little example we want to see Lemma 4.4 for the given graph in numbers. The graph is already defined in this Jupyter Notebook script at section 4.1. Since Node 4 of $G$ is globally reachable, we expect the 4th column of the sum of matrix powers $\sum_{k=0}^{n-1} A^k$ to be positive.

In [5]:
def sum_of_powers(M, n):
    """Returns the sum of the [0,n) powers of M"""
    result = np.zeros(M.shape)
    for i in range(n):
        result += np.linalg.matrix_power(M, i)
    return result

def is_irreducible(M):
    """Returns whether or not given square, positive matrix is irreducible"""
    Mk = sum_of_powers(M, M.shape[0])
    return not np.any(Mk == 0)

def is_node_globally_reachable(M, i):
    """Returns whether or not given node in given square, positive matrix is globally reachable"""
    power_sum = sum_of_powers(M, M.shape[0])
    return not np.any(power_sum[:,i] == 0) 

def is_primitive(M):
    """
    Returns whether of not a given square is primitive
    
    Corollary 8.5.8 in Horn & Johnson, Matrix Analysis:
    Let A be an n×n non-negative matrix. Then A is primitive if and only if A^(n2−2n+2) has only positive entries.
    """
    n = M.shape[0]
    return not np.any(np.linalg.matrix_power(M, n**2-2*n+2) == 0)

def draw_adj_matrix(A, ax):
    G = nx.DiGraph()
    for i in range(len(A)):
        for j in range(len(A[i])):
            if(A[i,j]):
                G.add_edge(i,j)
    nx.draw_networkx(G, node_size=100, ax=ax, connectionstyle='arc3, rad = 0.1')


In [6]:
n = np.size(A_di, 1)
p = 1.0/3
A_di_sum_of_powers = sum_of_powers(A_di, n-1)
print("4th column is positive, since it is the only globally reachable node:")
lib.matprint(A_di_sum_of_powers)

4th column is positive, since it is the only globally reachable node:
|   60.242  125.541  88.218   25.526   32.292  |
|  301.977    33.93   23.14   84.682   53.222  |
|   54.648   37.444  27.312   35.434   56.948  |
|        0        0       0        1        0  |
|  253.836   87.912  61.776  108.124  136.256  |


## 4.4. Graph theoretical characterization of primitive matrices

In this section we want to show pixel pictures of different matrix powers similar to figure 4.3. Remember, that (i) $G$ is strongly connected and aperiodic and (ii) $A$ is primitive are equivilant statements. We generate random examples by building the adjacency matrix first.

In [16]:
# Choose size of matrices here
n = 6

def plot_graph_and_matrix_powers(A, npwr=None):
    """ Function to visualize graph and matrix power of given matrix A only used in this Jupyter Notebook"""
    if npwr is None:
        npwr = A.shape[0]
    fig, ax = plt.subplots(figsize=custom_figsize)
    fig, axs = plt.subplots(1, npwr, figsize=(custom_figsize[0]*1.5, custom_figsize[1]/2))

    draw_adj_matrix(A, ax)

    for i in range(npwr):
        plot_matrix_binary(np.linalg.matrix_power(A, i+1), ax=axs[i])
        axs[i].set(title='${}^{{{}}}$'.format("M", i+1))

**Random Primitive Adjacency Matrix Example**

In [17]:
# Randomly generate primitive matrix
A_rand_prim = np.zeros((n, n))
while(not is_primitive(A_rand_prim)):
    A_rand_prim = np.random.choice([0, 1], size=(n,n), p=[1-p, p])

In [18]:
plot_graph_and_matrix_powers(A_rand_prim, npwr=n)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Random Irreducible  not Primitive Adjacency Matrix Example**

Note: Sometimes can take some time to generate a irreducible, but not positive matrix!

In [19]:
# Randomly generate irreducible, but not positive, matrix
A_rand_irr = np.zeros((n, n))
while(not is_irreducible(A_rand_irr) or is_primitive(A_rand_irr)):
    A_rand_irr = np.random.choice([0, 1], size=(n,n), p=[1-p, p])

In [20]:
plot_graph_and_matrix_powers(A_rand_irr, npwr=2*n)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Random Reducible Matrix Example**

In [21]:
# Randomly generate reducible adjacency matrix
A_rand_red = np.random.choice([0, 1], size=(n,n), p=[1-p, p])
while(is_irreducible(A_rand_red)):
    A_rand_red = np.random.choice([0, 1], size=(n,n), p=[1-p, p])

In [22]:
plot_graph_and_matrix_powers(A_rand_red, npwr=2*n)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …